In [1]:
import requests
from datetime import datetime
import pandas as pd

# Example usage
city = 'London'
date = '2025-04-01'


# Convert the date to the required format
date_obj = datetime.strptime(date, "%Y-%m-%d")
formatted_date = date_obj.strftime("%Y-%m-%d")

# Open-Meteo API endpoint for historical weather data
url = f"https://api.open-meteo.com/v1/forecast"

# Get the latitude and longitude of the city using Open-Meteo's geocoding API
geocode_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
response = requests.get(geocode_url)

if response.status_code == 200:
    location_data = response.json()
    if location_data and location_data['results']:
        lat = location_data['results'][0]['latitude']
        lon = location_data['results'][0]['longitude']
    else:
        print("City not found")        
else:        
    print("Error fetching location data")


# Parameters for the API request
params = {
    'latitude': lat,
    'longitude': lon,
    'start_date': formatted_date,
    'end_date': formatted_date,
    'hourly': 'temperature_2m',
    'timezone': 'auto'
}

# Make the API request
response = requests.get(url, params=params)

if response.status_code == 200:
    weather_data = response.json()
    hourly_data = weather_data['hourly']
    
    # Create a DataFrame from the hourly data
    df_weather = pd.DataFrame({
        'date': [formatted_date] * len(hourly_data['time']),
        'time': hourly_data['time'],
        'temperature': hourly_data['temperature_2m']
    })
    
    print(f"Weather data for {city} on {date}:")
    print(df_weather)
else:
    print("Error fetching weather data")


Weather data for London on 2025-04-01:
          date              time  temperature
0   2025-04-01  2025-04-01T00:00          7.9
1   2025-04-01  2025-04-01T01:00          7.6
2   2025-04-01  2025-04-01T02:00          7.3
3   2025-04-01  2025-04-01T03:00          7.0
4   2025-04-01  2025-04-01T04:00          6.9
5   2025-04-01  2025-04-01T05:00          6.7
6   2025-04-01  2025-04-01T06:00          6.8
7   2025-04-01  2025-04-01T07:00          7.3
8   2025-04-01  2025-04-01T08:00          7.4
9   2025-04-01  2025-04-01T09:00          8.3
10  2025-04-01  2025-04-01T10:00          9.4
11  2025-04-01  2025-04-01T11:00         10.1
12  2025-04-01  2025-04-01T12:00         11.4
13  2025-04-01  2025-04-01T13:00         12.9
14  2025-04-01  2025-04-01T14:00         13.0
15  2025-04-01  2025-04-01T15:00         13.0
16  2025-04-01  2025-04-01T16:00         13.1
17  2025-04-01  2025-04-01T17:00         12.8
18  2025-04-01  2025-04-01T18:00         12.1
19  2025-04-01  2025-04-01T19:00         

In [4]:
# Take existing Pandas Data Frame and Write to Lakehouse Table


import duckdb
from deltalake import write_deltalake
import pandas as pd
import notebookutils  # Fabric-specific utility
from datetime import datetime

# Define the Table Name for the Lakehouse
table_name = "weather_London"

# Define Data Frame Name
dataframe_name = "df_weather"

# Define the Table Mode Overwrite existing table; use "append" to add data
table_mode = "overwrite"

# Get the access token for Fabric authentication
access_token = notebookutils.credentials.getToken('storage')

# Define storage options for Fabric Lakehouse
storage_options = {
    "bearer_token": access_token,
    "use_fabric_endpoint": "true",
    "allow_unsafe_rename": "true"  # Useful for overwriting tables
}

# Define the path to the Lakehouse table (adjust to your workspace and lakehouse)
table_path = f"/lakehouse/default/Tables/{table_name}"

# Use DuckDB to query the DataFrame (optional SQL transformation step)
# Here, we just select all data, but you could add complex SQL logic
duckdb_result = duckdb.sql(f"SELECT * FROM {dataframe_name}").arrow()

# Write the result to a Delta table in the Lakehouse
write_deltalake(
    table_path,
    duckdb_result,
    mode=table_mode,  # Overwrite existing table; use "append" to add data
    engine="rust",     # Use the Rust engine for Delta writing
    storage_options=storage_options
)

print(f"Data successfully written to Lakehouse table! {table_path}")

rr /synfs/lakehouse/default/Tables/weather_London/_delta_log/_commit_20b063a9-7439-4076-b912-67436ef09516.json.tmp -> /synfs/lakehouse/default/Tables/weather_London/_delta_log/00000000000000000002.json
Data successfully written to Lakehouse table! /lakehouse/default/Tables/weather_London


In [22]:
# Converting Date or DateTime to UTC for Correct DateTime Data Type in Lakehouse
import pandas as pd
from datetime import datetime

df_dt_fixed = df_weather
df_dt_fixed['date'] = pd.to_datetime(df_dt_fixed["date"], utc=True)
df_dt_fixed['time'] = pd.to_datetime(df_dt_fixed["time"], utc=True)

display(df_dt_fixed)


In [23]:
# Take existing Pandas Data Frame and Write to Lakehouse Table
#### NOTE: This includes writing the data with the valid Date Time Column

import duckdb
from deltalake import write_deltalake
import pandas as pd
import notebookutils  # Fabric-specific utility
from datetime import datetime

# Define the Table Name for the Lakehouse
table_name = "weather_London_dt"

# Define Data Frame Name
dataframe_name = "df_dt_fixed"

# Define the Table Mode Overwrite existing table; use "append" to add data
table_mode = "overwrite"

# Get the access token for Fabric authentication
access_token = notebookutils.credentials.getToken('storage')

# Define storage options for Fabric Lakehouse
storage_options = {
    "bearer_token": access_token,
    "use_fabric_endpoint": "true",
    "allow_unsafe_rename": "true"  # Useful for overwriting tables
}

# Define the path to the Lakehouse table (adjust to your workspace and lakehouse)
table_path = f"/lakehouse/default/Tables/{table_name}"

# Use DuckDB to query the DataFrame (optional SQL transformation step)
# Here, we just select all data, but you could add complex SQL logic
duckdb_result = duckdb.sql(f"SELECT * FROM {dataframe_name}").arrow()

# Write the result to a Delta table in the Lakehouse
write_deltalake(
    table_path,
    duckdb_result,
    mode=table_mode,  # Overwrite existing table; use "append" to add data
    engine="rust",     # Use the Rust engine for Delta writing
    storage_options=storage_options
)

print(f"Data successfully written to Lakehouse table! {table_path}")

rr /synfs/lakehouse/default/Tables/weather_London_dt/_delta_log/_commit_01de0032-5431-47d3-9a36-06cf3735ffb4.json.tmp -> /synfs/lakehouse/default/Tables/weather_London_dt/_delta_log/00000000000000000000.json
Data successfully written to Lakehouse table!
